In [1]:
import ast
import glob
import warnings
from collections import defaultdict
from datetime import date

import numpy as np
import pandas as pd
import wandb

today = date.today()
api = wandb.Api()

# # Find all csv files in the current directory
csv_files = glob.glob("*.csv")
# # Collect all the names of the csv files without the extension
csv_names = [csv_file[:-4] for csv_file in csv_files]
project_name = "TopoBench"  #'best_results_edhnn'
user = "telyatnikov_sap"

if project_name not in csv_names:
    runs = api.runs(f"{user}/{project_name}")

    summary_list, config_list, name_list = [], [], []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k, v in run.config.items() if not k.startswith("_")}
        )

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    runs_df = pd.DataFrame(
        {"summary": summary_list, "config": config_list, "name": name_list}
    )

    runs_df.to_csv(f"{project_name}.csv")
else:
    runs_df = pd.read_csv(f"{project_name}.csv", index_col=0)

    for row in runs_df.iloc:
        row["summary"] = ast.literal_eval(row["summary"])
        row["config"] = ast.literal_eval(row["config"])


for row in runs_df.iloc:
    row["summary"].update(row["config"])

lst = [i["summary"] for i in runs_df.iloc]
df = pd.DataFrame.from_dict(lst)

df_init = df.copy()

# Get average epoch run time
df["epoch_run_time"] = df["_runtime"] / df["epoch"]

In [2]:
df_init.columns

Index(['AvgTime/train_batch_mean', 'AvgTime/train_batch_std',
       'AvgTime/train_epoch_mean', 'AvgTime/train_epoch_std',
       'AvgTime/val_batch_mean', 'AvgTime/val_batch_std',
       'AvgTime/val_epoch_mean', 'AvgTime/val_epoch_std', '_runtime', '_step',
       '_timestamp', '_wandb', 'epoch', 'lr-Adam', 'test/accuracy',
       'test/auroc', 'test/loss', 'test/precision', 'test/recall',
       'train/accuracy', 'train/auroc', 'train/loss', 'train/precision',
       'train/recall', 'trainer/global_step', 'val/accuracy', 'val/auroc',
       'val/loss', 'val/precision', 'val/recall', 'loss', 'seed', 'tags',
       'test', 'model', 'paths', 'train', 'extras', 'logger', 'dataset',
       'trainer', 'callbacks', 'ckpt_path', 'evaluator', 'optimizer',
       'task_name', 'transforms', 'model/params/total',
       'model/params/trainable', 'model/params/non_trainable', 'test/mae',
       'test/mse', 'train/mae', 'train/mse', 'val/mae', 'val/mse'],
      dtype='object')

In [3]:
def normalize_column(df, column_to_normalize):
    # Use json_normalize to flatten the nested dictionaries into separate columns
    flattened_df = pd.json_normalize(df[column_to_normalize])
    # Rename columns to include 'nested_column' prefix
    flattened_df.columns = [
        f"{column_to_normalize}.{col}" for col in flattened_df.columns
    ]
    # Concatenate the flattened DataFrame with the original DataFrame
    result_df = pd.concat([df, flattened_df], axis=1)
    # Get new columns names
    new_columns = flattened_df.columns
    # Drop the original nested column if needed
    result_df.drop(column_to_normalize, axis=1, inplace=True)
    return result_df, new_columns


# Config columns to normalize
columns_to_normalize = ["model", "dataset", "callbacks", "paths"]

# Keep track of config columns added
config_columns = []
for column in columns_to_normalize:
    df, columns = normalize_column(df, column)
    config_columns.extend(columns)

In [4]:
# Remove columns that are not needed (we shouldn't vary them or their variation is not interesting)
remove_col = [
    #"dataset.transforms.data_manipulations.selected_fields",
    "callbacks.model_checkpoint.dirpath", 'model.feature_encoder.selected_dimensions','callbacks.model_checkpoint.dirpath',
]
df = df.drop(remove_col, axis=1)

# Ensure that removed columns are not present in config_columns
config_columns = [col for col in config_columns if col != remove_col]

In [5]:
print(
    f"Number of rows with model.backbone._target_ = nan is {sum(df['model.backbone._target_'].isna())}"
)
# Drop na values if there are
df = df.dropna(subset=["model.backbone._target_"])
# Reset index
df = df.reset_index(drop=True)

# Drop rows that 'callbacks.early_stopping.monitor' isna
print(
    f"Number of rows with callbacks.early_stopping.monitor = nan is {sum(df['callbacks.early_stopping.monitor'].isna())}"
)

# print("Because of SCCN and CWN false runs there were 96 such runs on 13/03/24")

df = df.dropna(subset=["callbacks.early_stopping.monitor"])
# Reset index
df = df.reset_index(drop=True)


# Get correct names for the models
df["model.backbone._target_"] = df["model.backbone._target_"].apply(
    lambda x: x.split(".")[-1]
)

Number of rows with model.backbone._target_ = nan is 0
Number of rows with callbacks.early_stopping.monitor = nan is 0


In [6]:
df["model.backbone._target_"].unique()

array(['CCCN', 'CCXN', 'CWN', 'GAT', 'GCN', 'GIN', 'AllSetTransformer',
       'EDGNN', 'UniGCNII', 'SCCN', 'SCCNNCustom', 'SCN2'], dtype=object)

In [24]:
# Identify unique models in DataFrame
unique_models = df["model.backbone._target_"].unique()

# Identify unique datasets in DataFrame
unique_datasets = df["dataset.loader.parameters.data_name"].unique()


collected_results = defaultdict(dict)
collected_results_time = defaultdict(dict)
collected_results_time_run = defaultdict(dict)

collected_aggregated_results = defaultdict(dict)
collected_non_aggregated_results = defaultdict(dict)

# Got over each dataset and model and find the best result
for dataset in unique_datasets:
    for model in unique_models:
        # Get the subset of the DataFrame for the current dataset and model
        subset = df[
            (df["dataset.loader.parameters.data_name"] == dataset)
            & (df["model.backbone._target_"] == model)
        ]

        if subset.empty:
            print("---------")
            print(f"No results for {model} on {dataset}")
            print("---------")
            continue
        # Suppress all warnings
        warnings.filterwarnings("ignore")
        subset["Model"] = model
        warnings.filterwarnings("default")

        def get_metric(df):
            metric_ = df["callbacks.early_stopping.monitor"].unique()
            assert len(metric_) == 1, "There should be only one metric to optimize"
            metric = metric_[0]
            return metric.split("/")[-1]

        # Cols to get statistics later
        # TODO: log maximum validation value for optimized metric
        performance_cols = [f"test/{get_metric(subset)}"]

        # Get the unique values for each config column
        unique_colums_values = {}
        for col in config_columns:
            try:
                unique_colums_values[col] = subset[col].unique()
            except:
                print(f"Attention the columns: {col}, has issues with unique values")

        # Keep only those keys that have more than one unique value
        unique_colums_values = {
            k: v for k, v in unique_colums_values.items() if len(v) > 1
        }

        # Print the unique values for each config column

        print(f"Unique values for each config column for {model} on {dataset}:")
        for col, unique in unique_colums_values.items():
            print(f"{col}: {unique}")
            print()
        print("---------")

        # Check if "special colums" are not in unique_colums_values
        # For example dataset.parameters.data_seed should not be in aggregation columns
        # If it is, then we should remove it from the list
        special_columns = ["dataset.parameters.data_seed"]

        for col in special_columns:
            if col in unique_colums_values:
                unique_colums_values.pop(col)

        # Obtain the aggregation columns
        aggregation_columns = ["Model"] + list(unique_colums_values.keys())

        collected_non_aggregated_results[dataset][model] = {
            "df": subset.copy(),
            "aggregation_columns": aggregation_columns,
            "performance_cols": performance_cols,
        }

        # Aggregate the subset by the aggregation columns and get the best result for each group
        aggregated = subset.groupby(aggregation_columns).agg(
            {col: ["mean", "std"] for col in performance_cols}
        )

        # Go from MultiIndex to Index
        aggregated = aggregated.reset_index()

        assert (
            len(subset["callbacks.early_stopping.mode"].unique()) == 1
        ), "There should be only one mode for early stopping"
        # Identify the mode of the early stopping mode

        if subset["callbacks.early_stopping.mode"].unique()[0] == "max":
            ascending = False
            final_best_ = aggregated.sort_values(
                by=(f"test/{get_metric(subset)}", "mean"), ascending=ascending
            ).head(1)
            final_best_ = (final_best_ * 100).round(2)
        else:
            ascending = True
            final_best_ = aggregated.sort_values(
                by=(f"test/{get_metric(subset)}", "mean"), ascending=ascending
            ).head(1)

        collected_results[dataset][model] = {
            "mean": final_best_[(f"test/{get_metric(subset)}", "mean")].values[0],
            "std": final_best_[(f"test/{get_metric(subset)}", "std")].values[0],
        }

        # Get average epoch run time
        collected_results_time[dataset][model] = {
            "mean": subset['AvgTime/train_epoch_mean'].mean(),
            "std": subset["epoch"].std(),
        }

        collected_results_time_run[dataset][model] = {
            "mean": subset["_runtime"].mean(),
            "std": subset["_runtime"].std(),
        }

        collected_aggregated_results[dataset][model] = aggregated.sort_values(
            by=(f"test/{get_metric(subset)}", "mean"), ascending=ascending
        )

Attention the columns: model.feature_encoder.in_channels, has issues with unique values
Attention the columns: model.feature_encoder.selected_dimensions, has issues with unique values
Attention the columns: callbacks.model_checkpoint.dirpath, has issues with unique values
Unique values for each config column for CCCN on IMDB-BINARY:
dataset.split_params.data_seed: [0 3 5 7 9]

paths.output_dir: ['/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-08-10/0'
 '/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-08-10/1'
 '/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-08-10/2'
 '/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-08-10/3'
 '/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-08-10/4']

---------
Attention the columns: model.feature_encoder.in_channels, has issues with unique values
Attention the columns: model.feature_encoder.selected_dimensions, has issues with unique values
Attention the co

In [18]:
collected_non_aggregated_results.keys()

dict_keys(['IMDB-BINARY', 'IMDB-MULTI', 'MUTAG', 'NCI1', 'NCI109', 'PROTEINS', 'US-county-demos', 'amazon_ratings', 'citeseer', 'Cora', 'PubMed', 'minesweeper', 'roman_empire', 'REDDIT-BINARY', 'ZINC'])

In [ ]:
#"epoch_run_time" in collected_non_aggregated_results['NCI1']['CCCN']['df'].columns

True

### Runtime per epoch

In [20]:
collected_results_time
# Convert nested dictionary to DataFrame
nested_dict = dict(collected_results_time)
result_dict = pd.DataFrame.from_dict(
    {
        (i, j): nested_dict[i][j]
        for i in nested_dict
        for j in nested_dict[i].keys()
    },
    orient="index",
)


result_dict = result_dict.round(2)
result_dict["performance"] = result_dict.apply(
    lambda x: f"{x['mean']} ± {x['std']}", axis=1
)
result_dict = result_dict.drop(["mean", "std"], axis=1)

# Reset multiindex
result_dict = result_dict.reset_index()
# rename columns
result_dict.columns = ["Dataset", "Model", "Average Time per Epoch"]

In [ ]:
result_dict.pivot_table(
    index="Model", columns="Dataset", values="Average Time per Epoch", aggfunc="first"
)[['MUTAG', 'NCI1','NCI109','PROTEINS','ZINC','IMDB-BINARY','IMDB-MULTI']]

### Runtime

In [25]:
collected_results_time_run
# Convert nested dictionary to DataFrame
nested_dict = dict(collected_results_time_run)
result_dict = pd.DataFrame.from_dict(
    {
        (i, j): nested_dict[i][j]
        for i in nested_dict
        for j in nested_dict[i].keys()
    },
    orient="index",
)


result_dict = result_dict.round(2)
result_dict["performance"] = result_dict.apply(
    lambda x: f"{x['mean']} ± {x['std']}", axis=1
)
result_dict = result_dict.drop(["mean", "std"], axis=1)

# Reset multiindex
result_dict = result_dict.reset_index()
# rename columns
result_dict.columns = ["Dataset", "Model", "Average Training Time"]


In [26]:
result_dict.pivot_table(
    index="Model", columns="Dataset", values="Average Training Time", aggfunc="first"
)[['MUTAG', 'NCI1','NCI109','PROTEINS','ZINC','IMDB-BINARY','IMDB-MULTI']]

Dataset,MUTAG,NCI1,NCI109,PROTEINS,ZINC,IMDB-BINARY,IMDB-MULTI
Model,,,,,,,
AllSetTransformer,9.9 ± 3.24,138.13 ± 46.01,138.66 ± 26.38,15.81 ± 2.89,NaN,21.34 ± 2.96,20.89 ± 5.05
CCCN,12.04 ± 2.21,372.36 ± 109.47,272.3 ± 20.89,41.63 ± 7.23,1621.11 ± 141.72,51.88 ± 12.3,73.6 ± 20.07
CCXN,10.76 ± 1.89,244.72 ± 46.09,225.72 ± 86.57,51.98 ± 8.5,1226.67 ± 317.55,51.2 ± 18.14,72.3 ± 13.76
CWN,10.92 ± 0.96,302.34 ± 63.44,294.79 ± 46.27,53.6 ± 17.7,1390.21 ± 96.86,61.16 ± 9.33,39.99 ± 5.18
EDGNN,5.81 ± 1.13,110.86 ± 27.75,126.61 ± 45.53,15.15 ± 3.3,NaN,13.06 ± 4.76,13.89 ± 3.33
GAT,4.16 ± 1.05,57.32 ± 17.49,56.44 ± 9.05,8.18 ± 2.3,137.4 ± 35.13,7.48 ± 2.21,9.79 ± 1.85
GCN,3.83 ± 0.89,53.23 ± 19.67,37.4 ± 8.63,8.18 ± 2.47,NaN,8.18 ± 3.33,10.27 ± 3.58
GIN,4.6 ± 0.56,61.2 ± 23.97,50.32 ± 7.98,8.88 ± 2.34,NaN,7.72 ± 1.72,10.42 ± 3.93
SCCN,10.71 ± 2.92,332.76 ± 51.89,321.76 ± 55.92,70.06 ± 15.91,NaN,432.23 ± 57.29,716.44 ± 232.31
